## Języki symboliczne - rok akademicki 2022/2023

Przed rozpoczęciem pracy z notatnikiem zmień jego nazwę zgodnie z wzorem: `NrAlbumu_Nazwisko_Imie_PoprzedniaNazwa`

Przed wysłaniem notatnika **upewnij się jeszcze raz** że zmieniłeś nazwę i że rozwiązałeś wszystkie zadania/ćwiczenia, w szczególności, że uzupełniłeś wszystkie pola `YOUR CODE HERE` oraz `YOUR ANSWER HERE`.

# Temat: Klasy - iteratory, generatory, wyrażenia generatora.
Zapoznaj się z treścią niniejszego notatnika czytając i wykonując go komórka po komórce. Wykonaj napotkane zadania/ćwiczenia.


## Iteratory

https://docs.python.org/3/tutorial/classes.html#Iterators

- po większości obiektów będących kontenerami możemy iterować używając pętli `for`;
- funkcjonalność ta realizowana jest poprzez wywołanie przez `for` metody `iter()` na obiekcie kontenera, która zwraca obiekt iteratora definiujący metodę `__next__()` udostępniającą raz po razie kolejne elementy kontenera;
- w przypadku gdy nie ma już kolejnych elementów `__next__()` zgłasza wyjątek `StopIteration` powiadamiajacy pętlę `for`, że należy ją zakończyć;
- `__next__()` może być wywołane poprzez funkcję wbudowaną `next()`;



In [1]:
for element in 'abcd':
    print(element, end = ' ')

a b c d 

In [3]:
s = 'abcd'
it = iter(s)     # wywołanie metody iter() na obiekcie kontenera (str) zwraca obiekt iteratora definujący metodę __next__()
print(it)
print(next(it))  # wywołuje metodę __next__()
print(next(it))
print(next(it))
print(next(it))
next(it)          # zgłoszenie wyjątku StopIteration

a
b
c
d


StopIteration: 

Znając mechanizmy iteratora stosunkowo łatwo jest go zaimplementować w swoich klasach.

- należy zdefiniować metodę `__iter__()` zwracającą obiekt z metodą `__next__()`.
- Jeśli klasa definiuje `__next__()` to `__iter__()` zwraca po prostu `self`.

__Przykład__. Napisz klasę`Wspak` przyjmującą jako argument konstruktora tekst. Klasa ta powinna pełnić rolę iteratora, iterującego po podanym tekście wspak.

In [6]:
class Wspak:
    def __init__(self, text):
        self.text = text
        self.index = len(text)

    def __iter__(self):
        return self

    def __next__(self):
        if self.index == 0:
            raise StopIteration
        self.index = self.index - 1
        return self.text[self.index]

In [7]:
wsp = Wspak('spam') # konkretyzacja klasy  - iterator
print(wsp)
print(type(wsp))
for znak in wsp:
    print(znak)

<class '__main__.Wspak'>
m
a
p
s


In [8]:
next(wsp) # zgłasza wyjątek StopIteration

StopIteration: 

In [9]:
wsp1 = Wspak('maps')
wsp1.__next__(), wsp1.__next__(), wsp1.__next__(), wsp1.__next__() # wywołanie metody __next__()
#wsp1.__next__() # to wywołanie metody __next__() zgłasza wyjątek StopIteration

('s', 'p', 'a', 'm')

## Generatory

https://docs.python.org/3/tutorial/classes.html#generators

W Pythonie mamy dwa rodzaje generatorów:
- tworzone przez funkcje generujące;
- tworzone przez wyrażenia generujące.


Generatory można traktować jak funkcje, których działanie można wstrzymywać i wznawiać (zmienne lokalne nie są niszczone po opuszczeniu generatora umożliwiając późniejsze wznowienie w tym samym miejscu):
- Na potrzeby generatorów zostało wprowadzone nowe słowo kluczowe `yield`.
- Każda funkcja, która zawiera instrukcję `yield` staje się automatycznie funkcją generującą.
- Przy wywołaniu funkcji generującej nie jest zwracana pojedyncza wartość. Zamiast tego zwracany jest obiekt generatora, który obsługuje protokół iteratora.
- instrukcja `yield` występuje zamiast instrukcji `return`.  Różnica pomiędzy tymi instrukcjami polega jednak na tym, że w przypadku instrukcji `yield` zostaje zapamiętany stan wykonywania generatora oraz wartości wszystkich zmiennych lokalnych. Przy kolejnym wywołaniu metody `__next()__` generatora wykonywanie funkcji jest wznawiane bezpośrednio po ostatnio napotkanej instrukcji `yield`.




In [10]:
def wygeneruj_calkowite(N):
    for i in range(N):
        yield i              # yield oznacza, że jest to generator - nie zwykła funkcja

g = wygeneruj_calkowite(10); # funkcja wygeneruj_calkowite(10) zwraca obiekt generatora
print(g)

for j in g:
    print(j, end = ' ')

<generator object wygeneruj_calkowite at 0x7bba82364510>
0 1 2 3 4 5 6 7 8 9 

- Generator (funkcja realizująca generator) generuje i zwraca za każdym razem (z każdym wywołaniem funkcji) jedną wartość.
- Wywołanie po raz pierwszy metody generatora `__next__()` dla obiektu generatora powoduje wykonanie kodu w `utworz_licznik` do pierwszego wystąpienia `yield`, a następnie zwrócenie wydobytej wartości (w przykładzie poniżej 10).
- Kolejne wywołania `__next__()` rozpoczynają od miejsca ostatniego wyjścia z funkcji i działają do kolejnego napotkania `yield`.

In [11]:
def utworz_licznik(x):
    print('Wejście do licznika')
    while True:
        yield x                   # yield oznacza, że jest to generator
        print('zwiększenie x')
        x = x + 1

l = utworz_licznik(10)            # funkcja utworz_licznik(10) zwraca obiekt generatora
print(l)

next(l), next(l), next(l)

<generator object utworz_licznik at 0x7bba8020c040>
Wejście do licznika
zwiększenie x
zwiększenie x


(10, 11, 12)

- wszystko co może być zrobione za pomocą generatorów można zaimplementować również za pomocą iteratorów bazujących na klasach.
- to co wpływa na kompaktowość generatorów to to, że metody `__iter__()` oraz `__next__()` są tworzone automatycznie
- kolejną, kluczową ich cechą jest pamiętanie lokalnych zmiennych i stanu wykonywanego kodu pomiędzy kolejnymi do niego odwołaniami.
- to upraszcza implementację w stosunku do podejścia w którym używa się `self.index` i `self.data`.
- w przypadku zakończenia działania generatora zgłasza on automatycznie wyjątek `StopIteration`.

In [12]:
# generator
'''
def utworz(N):
    for i in range(N):
        yield i
'''
# iterator
class Utworz_1:
    def __init__(self, data):
        self.data = data
        self.index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.index == self.data:
            raise StopIteration
        self.index = self.index + 1
        return self.index

In [13]:
gg = Utworz_1(10); # konkretyzacja klasy  - iterator
print(gg)

for j in gg:
    print(j, end = ' ')

1 2 3 4 5 6 7 8 9 10 

## Wyrażenia generujące (`generator expressions`)

https://docs.python.org/3/tutorial/classes.html#generator-expressions


- Niektóre proste generatory mogą być efektywnie implementowane jako wyrażenia przy pomocy mechanizmu typu `list comprehension`, ale zamiast nawiasów kwadratowych `[]` używają nawiasów okrągłych `()`.

- Wyrażenia te przeznaczone są dla sytuacji w których generator jest użyty bezpośrednio w wywołującej go funkcji.


In [14]:
g = (i**2 for i in range(10))
print(g)
for j in g:
    print(j, end = ' ')

<generator object <genexpr> at 0x7bba8020c270>
0 1 4 9 16 25 36 49 64 81 

Wyrażenia generatora często są używane z takimi funkcjami jak `sum`, `min`, `max`.

In [15]:
sum(i**2 for i in range(10)) # użycie generatora bezpośrednio w funkcji sum()

285

## Przestrzenie nazw. Polecenia `nonlocal`, `global`.

- Przestrzenie nazw tworzone są w różnych chwilach i są aktywne przez różny czas.


- Przestrzeń nazw zawierająca nazwy wbudowane tworzona jest podczas rozpoczęcia pracy interpretera Pythona i nigdy nie jest usuwana. Nazwy wbudowane przechowywane są w module o nazwie `builtins`.


- Przestrzeń nazw globalnych modułu tworzona jest podczas wczytywania jego definicji i jest aktywna również do chwili zakończenia pracy interpretera.


- Instrukcje wykonywane przez wywołania interpretera, zarówno czytane z pliku jak i wprowadzane interaktywnie, są częścią modułu o nazwie `__main__` - tak więc posiadają swoją własną przestrzeń nazw globalnych.


- Przestrzeń nazw lokalnych funkcji tworzona jest w momencie jej wywołania i niszczona, gdy następuje powrót z funkcji lub zgłoszony został w niej wyjątek, który nie został tam obsłużony.


- Wywołanie rekurencyjne powoduje tworzenie za każdym razem nowej przestrzeni nazw lokalnych.


- W każdym momencie wykonania programu, istnieją co najmniej trzy zagnieżdżone zasięgi nazw (tzn. wprost osiągalne są trzy przestrzenie nazw):
  - najbardziej zagnieżdżony, w którym najpierw poszukuje się nazwy, zawiera on nazwy lokalne;
  - środkowy, przeszukiwany w następnej kolejności, który zawiera aktualne nazwy globalne modułu;
  - zewnętrzny (przeszukiwany na końcu) jest zasięgiem nazw wbudowanych.


__`globals()`__ -  zwraca słownik reprezentujący bieżącą globalną tablicę symboli. Jest to zawsze słownik bieżącego modułu (wewnątrz funkcji lub metody jest to moduł, w którym jest zdefiniowany, a nie moduł, z którego jest wywoływany).

__`locals()`__ - zwraca słownik reprezentujący bieżącą lokalną tablicę symboli. Na poziomie modułu `locals()` i `globals()` są tym samym słownikiem.



In [16]:
# Przed każdym uruchomieniem naciśnij: Kernel / Restart & clear output

def scope_test():

    def do_local():
        spam = "test_3"
        print('3)\n', locals())

    spam = "test_2"
    print('2)\n', locals())
    do_local()

spam = 'test_1'
print('1)\n', locals())
scope_test()

locals() == globals()

#dir(__builtins__) # zwraca nazwy zdefiniowane w module builtins

1)
 {'__name__': '__main__', '__doc__': '\ndef utworz(N):\n    for i in range(N):\n        yield i \n', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', "for element in 'abcd':\n    print(element, end = ' ')", "s = 'abcd'\nit = iter(s)     # wywołanie metody iter() na obiekcie kontenera (str) zwraca obiekt iteratora definujący metodę __next__()\nprint(it)\nprint(next(it))  # wywołuje metodę __next__() \nprint(next(it))\nprint(next(it))\nprint(next(it))\nnext(it)          # zgłoszenie wyjątku StopIteration ", "s = 'abcd'\nit = iter(s)     # wywołanie metody iter() na obiekcie kontenera (str) zwraca obiekt iteratora definujący metodę __next__()\nprint(it)\nprint(next(it))  # wywołuje metodę __next__() \nprint(next(it))\nprint(next(it))\nprint(next(it))\nnext(it)          # zgłoszenie wyjątku StopIteration ", 'next(wsp) # zgłasza wyjątek StopIteration ', '# YOUR CODE HERE\n

True

### Deklaracje `nonlocal`, `global`.

- deklaracji `nonlocal` używamy aby odwołać się do zmiennych znajdujących się poza najbardziej zagnieżdżonym zasięgiem. Bez niej zmienne te byłyby tylko do odczytu. Podczas zapisu tworzona by była zmienna o tej samej nazwie, a ta zewnętrzna pozostała by niezmieniona.


- deklaracja `global` może zostać użyta do oznaczenia, że wyszczególniona nazwa należy do przestrzeni nazw globalnych


- Zasięgi nazw zdeterminowane są przez ich zasięg w tekście. Zasięg globalny funkcji zdefiniowanej w danym module jest zasięgiem związanym z tym modułem, niezależnie gdzie i jak (przy pomocy jakiego aliasu funkcja jest wywoływana)


- Jedną z cech szczególnych Pythona jest to, że przypisanie zawsze zachodzi w najbardziej zagnieżdżonym zasięgu. Przypisania nie powodują kopiowania danych - przywiązują jedynie nazwy do obiektów.


- To samo zachodzi w przypadku usuwania: instrukcja `del x` usuwa związek obiektu identyfikowanego przez nazwę x z tą nazwą w przestrzeni nazw lokalnych

In [17]:
def scope_test():
    def do_local():
        spam = "local spam"     # lokalne związanie nie zmienia spam w scope_test()

    def do_nonlocal():
        nonlocal spam           # nonlocal zmienia spam w scope_test()
        spam = "nonlocal spam"

    def do_global():
        global spam             # global zmienia spam na poziomie modułu
        spam = "global spam"

    spam = "test spam"
    do_local()
    print("After local assignment:", spam)
    do_nonlocal()
    print("After nonlocal assignment:", spam)
    do_global()
    print("After global assignment:", spam)

scope_test()
print("In global scope:", spam)

After local assignment: test spam
After nonlocal assignment: nonlocal spam
After global assignment: nonlocal spam
In global scope: global spam


## Zadanie 1

Napisz klasę`SamogloskaUpss` przyjmującą jako argument konstruktora tekst. Klasa ta powinna pełnić rolę iteratora, iterującego po podanym tekście. W przypadku napotkania samogłoski iterator zwraca tekst `Upss`.

https://docs.python.org/3/tutorial/classes.html#iterators

In [18]:
class SamogloskaUpss:
    def __init__(self, tekst):
        self.tekst = tekst
        self.index = 0
        self.samogloski = "aeiouyAEIOUY"  # samogłoski w języku polskim i angielskim

    def __iter__(self):
        return self

    def __next__(self):
        if self.index >= len(self.tekst):
            raise StopIteration

        char = self.tekst[self.index]
        if char in self.samogloski:
            self.index += 1
            return "Upss"

        self.index += 1
        return char

# Przykład użycia:
if __name__ == "__main__":
    tekst = "To jest przykładowy tekst"
    iterator = SamogloskaUpss(tekst)

    for element in iterator:
        print(element)


T
Upss
 
j
Upss
s
t
 
p
r
z
Upss
k
ł
Upss
d
Upss
w
Upss
 
t
Upss
k
s
t


## Zadanie 2
Napisz klasę `Unikalne` przyjmującą jako argument konstruktora pewną listę. Klasa ta powinna pełnić rolę iteratora, iterującego po unikalnych (występujących dokładnie jeden raz) elementach listy.


In [19]:
class Unikalne:
    def __init__(self, lista):
        self.lista = lista
        self.indeks = 0
        self.licznik = {}
        self._znajdz_unikalne()

    def _znajdz_unikalne(self):
        for element in self.lista:
            if element in self.licznik:
                self.licznik[element] += 1
            else:
                self.licznik[element] = 1

        # Znajdowanie unikalnych elementów
        self.unikalne_elementy = [element for element in self.lista if self.licznik[element] == 1]

    def __iter__(self):
        return self

    def __next__(self):
        if self.indeks >= len(self.unikalne_elementy):
            raise StopIteration

        element = self.unikalne_elementy[self.indeks]
        self.indeks += 1
        return element

# Przykład użycia:
if __name__ == "__main__":
    lista = [1, 2, 2, 3, 4, 5, 5, 6, 7, 7]
    iterator = Unikalne(lista)

    for element in iterator:
        print(element)


1
3
4
6


## Zadanie 3
Napisz funkcję `liczby_pierwsze` przyjmującą jako argument liczbę całkowitą. Funkcja ta powinna być generatorem zwracającym liczby pierwsze od 2 do liczby przesłanej jako argument, bez niej. Wykorzystaj słowo kluczowe `yield`.

Przykładowe użycie funkcji `liczby_pierwsze`:


```python
for i in liczby_pierwsze(100):
    print(i)
# powyższy kod powinien wypisać liczby pierwsze do liczby 97 włącznie
```

https://docs.python.org/3/tutorial/classes.html#generators

In [20]:
def liczby_pierwsze(n):
    """ Generator yielding prime numbers up to n (inclusive). """
    if n < 2:
        return

    # Inicjalizacja listy oznaczającej czy liczba jest pierwsza (True) czy nie (False)
    is_prime = [True] * (n + 1)
    is_prime[0] = is_prime[1] = False  # 0 i 1 nie są liczbami pierwszymi

    # Implementacja Sita Eratostenesa
    for start in range(2, int(n**0.5) + 1):
        if is_prime[start]:
            for multiple in range(start*start, n + 1, start):
                is_prime[multiple] = False

    # Generowanie liczb pierwszych
    for number in range(2, n + 1):
        if is_prime[number]:
            yield number

# Przykładowe użycie funkcji liczby_pierwsze:
if __name__ == "__main__":
    limit = 100
    for prime in liczby_pierwsze(limit):
        print(prime)


2
3
5
7
11
13
17
19
23
29
31
37
41
43
47
53
59
61
67
71
73
79
83
89
97


## Zadanie 4
Napisz funkcję `JednorekiBandyta`, przyjmującą jako argument pewną kwotę w groszach. Funkcja ta powinna być generatorem tworzącym obiekty monet o nominałach 1 lub 2 grosze, do kwoty zadanej argumentem. Klasa `Moneta` z poprzednich laboratoriów.

Przykładowo, wywołanie `JednorekiBandyta(100)` powinno wygenerować na przykład 100 monet 1gr, lub 50 monet 2gr, lub dowolną inną kombinację dającą w sumie 100gr.


In [21]:
class Moneta:
    def __init__(self, nominal):
        self.nominal = nominal  # nominal w groszach

    def __repr__(self):
        return f"Moneta({self.nominal}gr)"

def JednorekiBandyta(kwota_grosze):
    """ Generator yielding coins (Moneta objects) totaling up to kwota_grosze. """
    if kwota_grosze < 0:
        return

    while kwota_grosze > 0:
        if kwota_grosze >= 2:
            yield Moneta(2)
            kwota_grosze -= 2
        else:
            yield Moneta(1)
            kwota_grosze -= 1

# Przykładowe użycie funkcji JednorekiBandyta:
if __name__ == "__main__":
    kwota = 100
    for moneta in JednorekiBandyta(kwota):
        print(moneta)


Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)
Moneta(2gr)


## Zadanie 5

Do klasy `PrzechowywaczMonet` z poprzednich laboratoriów dodaj metodę `wszystkieMonety` będącą generatorem zwracającym kolejno wszystkie przechowywane monety posortowane rosnąco po nominale.


In [22]:
class Moneta:
    def __init__(self, nominal):
        self.nominal = nominal

    def __repr__(self):
        return f"Moneta({self.nominal}gr)"

class PrzechowywaczMonet:
    def __init__(self):
        self.monety = []

    def dodajMonete(self, moneta):
        self.monety.append(moneta)

    def wszystkieMonety(self):
        """ Generator yielding all stored coins sorted by nominal ascending. """
        sorted_monety = sorted(self.monety, key=lambda moneta: moneta.nominal)
        for moneta in sorted_monety:
            yield moneta

# Przykład użycia:
if __name__ == "__main__":
    przechowywacz = PrzechowywaczMonet()
    przechowywacz.dodajMonete(Moneta(5))
    przechowywacz.dodajMonete(Moneta(2))
    przechowywacz.dodajMonete(Moneta(10))

    for moneta in przechowywacz.wszystkieMonety():
        print(moneta)


Moneta(2gr)
Moneta(5gr)
Moneta(10gr)


## Zadanie 6
Napisz funkcję liczącą sumę iloczynów elementów dwóch list `a` i `b`:

`a[0]*b[0]+a[1]*b[1]+....`

Obie listy powinny być równej długości. Wykorzystaj `generator expressions` oraz funkcję `zip`.

https://docs.python.org/3/tutorial/classes.html#generator-expressions

In [23]:
def suma_iloczynow(a, b):
    """ Funkcja zwracająca sumę iloczynów elementów list a i b. """
    if len(a) != len(b):
        raise ValueError("Listy a i b muszą być równej długości.")

    suma = sum(a_i * b_i for a_i, b_i in zip(a, b))
    return suma

# Przykładowe użycie:
if __name__ == "__main__":
    lista_a = [1, 2, 3, 4]
    lista_b = [5, 6, 7, 8]

    wynik = suma_iloczynow(lista_a, lista_b)
    print("Suma iloczynów:", wynik)


Suma iloczynów: 70


## Zadanie 7

Napisz funkcję `srednia` będącą generatorem, liczącym średnią arytmetyczną wartości przesłanych do niej przy pomocy metody `send`.

Przykładowy kod używający `send`:

```python
def sumowanie():
    suma=0
    while True:
        a=(yield)
        print("Otrzymano:", a)
        suma+=a
        yield suma

generator=sumowanie()

for i in [1, 2, 3, 5, 7, 11]:
    next(generator)
    print("Generator zwrócił:", generator.send(i))
```



https://docs.python.org/3/reference/expressions.html#yield-expressions

In [24]:
def srednia():
    suma = 0
    licznik = 0
    while True:
        wartosc = yield
        print("Otrzymano:", wartosc)
        suma += wartosc
        licznik += 1
        srednia = suma / licznik
        yield srednia

# Przykładowe użycie:
if __name__ == "__main__":
    generator = srednia()

    for i in [1, 2, 3, 5, 7, 11]:
        next(generator)  # Przygotowanie generatora do odbioru wartości
        print("Generator zwrócił:", generator.send(i))


Otrzymano: 1
Generator zwrócił: 1.0
Otrzymano: 2
Generator zwrócił: 1.5
Otrzymano: 3
Generator zwrócił: 2.0
Otrzymano: 5
Generator zwrócił: 2.75
Otrzymano: 7
Generator zwrócił: 3.6
Otrzymano: 11
Generator zwrócił: 4.833333333333333


## Zadanie 8

Dany jest kod:

```python
x=0

def bla():
    x+=1
    
print(x)
bla()
print(x)
```

Zmodyfikuj funkcję `bla` tak, aby zwiększała wartość zmiennej globalnej `x` o 1.

Wykorzystaj słowo kluczowe `global`.



In [25]:
x = 0

def bla():
    global x
    x += 1

print(x)  # Wyświetli: 0
bla()
print(x)  # Wyświetli: 1


0
1


## Zadanie 9

Dany jest kod:

```python
x=1337

def foo():
    def bar():
        x+=1
    x=0
    print(x)
    bar()
    print(x)
    
print(x)
foo()
print(x)

```

Zmodyfikuj funkcje `foo` i `bar` tak, aby funkcja `bar` zwiększała zmienną `x` zdefiniowaną w funkcji `foo` o 1, nie zmieniając wartości zmiennej globalnej `x`.

Wykorzystaj słowo kluczowe `nonlocal`.

In [26]:
x = 1337

def foo():
    x = 0

    def bar():
        nonlocal x
        x += 1

    print(x)  # Wyświetli: 0
    bar()
    print(x)  # Wyświetli: 1

print(x)  # Wyświetli: 1337
foo()
print(x)  # Wyświetli: 1337


1337
0
1
1337
